In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import sys
sys.path.append('custom_modules')

import Data_processing as dp

from IPython.display import display
from matplotlib import ticker
from pathlib import Path
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Reshape, Input, Dense, Flatten, Conv2D, Dropout, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D, concatenate, BatchNormalization

### Константы для путей к файлам

In [2]:
# Пути к обработанным данным
# Пути к X части выборки с массивами в каждой ячейке
path_to_array_like_X_data = ('data/Prepared data/Run1/X_data_array_like.xlsx',
                             'data/Prepared data/Run2/X_data_array_like.xlsx')
# Пути к Y части выборки
path_to_Y_data = ('data/Prepared data/Run1/Y_data(binary_classification).xlsx', 
                  'data/Prepared data/Run2/Y_data(binary_classification).xlsx') 


### Константы для обработки данных

In [3]:
PREP_image_size = 64
PREP_crop_step = 3

In [4]:
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

# Подготовка данных

In [5]:
# загрузка данных из файлов
X_dict = dict()
Y_dict = dict()

for i,x_data_path,y_data_path in zip(range(len(path_to_array_like_X_data)),
                                     path_to_array_like_X_data, 
                                     path_to_Y_data):
    
    x_temp = dp.get_array_like_X_df(x_data_path)
    y_temp = dp.get_Y_df(y_data_path)
    
    
    
    X_dict[f'run{i+1}'] = {'df' : x_temp.iloc[:,int(x_temp.shape[1]/2):]}
    Y_dict[f'run{i+1}'] = {'df' : y_temp.iloc[:,int(y_temp.shape[1]/2):]}

In [6]:
print('Загружены данные для X выборки из файлов: ', X_dict.keys())
print('Загружены данные для Y выборки из файлов: ', Y_dict.keys())

Загружены данные для X выборки из файлов:  dict_keys(['run1', 'run2'])
Загружены данные для Y выборки из файлов:  dict_keys(['run1', 'run2'])


In [ ]:
fig, axes = plt.subplots(2)

fig.set_figwidth(18)
fig.set_figheight(10)

for ax,key in zip(axes,Y_dict.keys()):
    ax.pcolormesh(Y_dict[key]['df'])

    ax.set_xlabel('Номер датчика', fontsize=10) 
    ax.set_ylabel('Номер измерения', fontsize=10) 
    ax.set_title(f'Часть начального датафрейма для тренировки модели (файл: {key})', fontsize=10) 
     
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.1, hspace=0.2)
plt.show()

In [7]:
# добавление строк в датафреймы
for i,key in zip(range(len(X_dict.keys())),X_dict.keys()):
    print(f"||||||||||| step:  {i} |||||||||||")
    print('||||||||||| X df preprocessing |||||||||||')
    X_dict[key]['df'] = dp.reshape_df_for_future_crops(X_dict[key]['df'], 
                                                        PREP_image_size, 
                                                        PREP_crop_step)
    print('||||||||||| Y df preprocessing |||||||||||')
    Y_dict[key]['df'] = dp.reshape_df_for_future_crops(Y_dict[key]['df'], 
                                                        PREP_image_size, 
                                                        PREP_crop_step) 


    print('\n')

||||||||||| step:  0 |||||||||||
||||||||||| X df preprocessing |||||||||||
||||||||||||||||||
Df reshaping for exact splitting with crop_size
Original df size:  (115, 200)
Crop windows height/width:  64
Crop windows step across rows and cols:  2
New df shape:  (116, 200)
||||||||||||||||||

||||||||||| Y df preprocessing |||||||||||
||||||||||||||||||
Df reshaping for exact splitting with crop_size
Original df size:  (115, 200)
Crop windows height/width:  64
Crop windows step across rows and cols:  2
New df shape:  (116, 200)
||||||||||||||||||



||||||||||| step:  1 |||||||||||
||||||||||| X df preprocessing |||||||||||
||||||||||||||||||
Df reshaping for exact splitting with crop_size
Original df size:  (119, 200)
Crop windows height/width:  64
Crop windows step across rows and cols:  2
New df shape:  (120, 200)
||||||||||||||||||

||||||||||| Y df preprocessing |||||||||||
||||||||||||||||||
Df reshaping for exact splitting with crop_size
Original df size:  (119, 200)
Crop windows

In [8]:
# запишем в словарь numpy array полученные из df
for i,key in zip(range(len(X_dict.keys())),X_dict.keys()):
    print(f"|||||||||||||| step: {i+1}||||||||||||||||")
    print(f"|||||||||||||| file: {key}||||||||||||||||")
    print('||||||||||| X df preprocessing |||||||||||')
    x_time_temp, \
    x_amp_temp = dp.reshape_X_df_to_image_like_numpy(X_dict[key]['df'], 
                                                        PREP_image_size, 
                                                        PREP_crop_step)
    X_dict[key]['X_time_all'] = x_time_temp.astype('float32')
    X_dict[key]['X_amp_all'] = x_amp_temp.astype('float32')

    X_dict[key].pop('df',0)
    
    print('||||||||||| Y df preprocessing |||||||||||')
    y_temp = dp.reshape_Y_df_to_image_like_numpy(Y_dict[key]['df'], 
                                                 PREP_image_size, 
                                                 PREP_crop_step)
    Y_dict[key]['Y_mask_all'] = y_temp.astype('int8')
    
    Y_dict[key].pop('df',0)
    
    print('\n')
    
    

|||||||||||||| step: 1||||||||||||||||
|||||||||||||| file: run1||||||||||||||||
||||||||||| X df preprocessing |||||||||||
||||||||||||||||||
X df reshaping to 4D
Original df size:  (116, 200)
Crop windows height/width:  64
Crop windows step across rows and cols:  2
New X_time shape:  (1863, 64, 64, 32)
New X_amp shape:  (1863, 64, 64, 32)
||||||||||||||||||

||||||||||| Y df preprocessing |||||||||||
||||||||||||||||||
Y df reshaping to 3D
Original df size:  (116, 200)
Crop windows height/width:  64
Crop windows step across rows and cols:  2
New numpy shape:  (1863, 64, 64, 1)
||||||||||||||||||



|||||||||||||| step: 2||||||||||||||||
|||||||||||||| file: run2||||||||||||||||
||||||||||| X df preprocessing |||||||||||
||||||||||||||||||
X df reshaping to 4D
Original df size:  (120, 200)
Crop windows height/width:  64
Crop windows step across rows and cols:  2
New X_time shape:  (2001, 64, 64, 32)
New X_amp shape:  (2001, 64, 64, 32)
||||||||||||||||||

||||||||||| Y df preprocessin

In [9]:
print('||||||||||||||||||||||||||||')
print('Данные считаны для файлов: ',X_dict.keys())
print('\n||||||||||| X data |||||||||||')
for key in X_dict.keys():
    print(f'Для файла \"{key}\" для \"X\" выборки считано ', X_dict[key].keys())

print('\n||||||||||| Y data |||||||||||')        
for key in Y_dict.keys():
    print(f'Для файла \"{key}\" для \"Y\" выборки считано ', Y_dict[key].keys())

||||||||||||||||||||||||||||
Данные считаны для файлов:  dict_keys(['run1', 'run2'])

||||||||||| X data |||||||||||
Для файла "run1" для "X" выборки считано  dict_keys(['X_time_all', 'X_amp_all'])
Для файла "run2" для "X" выборки считано  dict_keys(['X_time_all', 'X_amp_all'])

||||||||||| Y data |||||||||||
Для файла "run1" для "Y" выборки считано  dict_keys(['Y_mask_all'])
Для файла "run2" для "Y" выборки считано  dict_keys(['Y_mask_all'])


In [10]:
# обработка данных
for i,key in zip(range(len(X_dict.keys())),X_dict.keys()):
    print(f"|||||||||||||| step: {i+1}||||||||||||||||")
    print(f"|||||||||||||| file: {key}||||||||||||||||")
    print('||||||||||| X df preprocessing |||||||||||')

    (X_time_def, X_time_non_def), \
    (X_amp_def, X_amp_non_def), \
    (Y_mask_def, Y_mask_non_def), \
    (Y_binary_def, Y_binary_non_def) = dp.preprocess_data(X_dict[key]['X_time_all'], 
                                                                    X_dict[key]['X_amp_all'], 
                                                                    Y_dict[key]['Y_mask_all'],
                                                                    PREP_image_size)

    X_dict[key]['X_time_def'] = X_time_def
    X_dict[key]['X_time_non_def'] = X_time_non_def

    X_dict[key]['X_amp_def'] = X_amp_def
    X_dict[key]['X_amp_non_def'] = X_amp_non_def

    Y_dict[key]['Y_binary_def'] = Y_binary_def
    Y_dict[key]['Y_binary_non_def'] = Y_binary_non_def

|||||||||||||| step: 1||||||||||||||||
|||||||||||||| file: run1||||||||||||||||
||||||||||| X df preprocessing |||||||||||
||||||||||||||||||
Data preprocessing
Orig X_time shape:  (1863, 64, 64, 32)
Orig X_amp shape:  (1863, 64, 64, 32)
Orig Y_mask shape:  (1863, 64, 64, 1)
||||||||||||||||||
Data standartization
X_time_max: 44.5
X_amp_max: 62.992000579833984

X_time_max after standartization: 1.0
X_time_min after standartization: 0.0

X_amp_max after standartization: 1.0
X_amp_min after standartization: -0.9837439060211182
||||||||||||||||||

||||||||||||||||||
Data with and witout defects splitting
||||||||||||||||||
Defects nums calculating
Для карт высотой и шириной в 64
и общим кличеством: 1863
дефекты присутствуеют на 513 картах
||||||||||||||||||

X_time_def shape:  (513, 64, 64, 32)
X_time_non_def shape:  (1350, 64, 64, 32)

X_amp_def shape:  (513, 64, 64, 32)
X_amp_non_def shape:  (1350, 64, 64, 32)

Y_mask_def shape:  (513, 64, 64, 1)
Y_mask_non_def shape:  (1350, 64, 64, 1

In [11]:
# очистка памяти от лишнего
for key in X_dict.keys():
    X_dict[key].pop('X_time_all',0)
    X_dict[key].pop('X_amp_all',0)
    Y_dict[key].pop('Y_mask_all',0)

In [12]:
print('||||||||||||||||||||||||||||')
print('Данные считаны для файлов: ',X_dict.keys())
print('\n||||||||||| X data |||||||||||')
for key in X_dict.keys():
    print(f'Для файла \"{key}\" для \"X\" выборки считано ', X_dict[key].keys())

print('\n||||||||||| Y data |||||||||||')        
for key in Y_dict.keys():
    print(f'Для файла \"{key}\" для \"Y\" выборки считано ', Y_dict[key].keys())

||||||||||||||||||||||||||||
Данные считаны для файлов:  dict_keys(['run1', 'run2'])

||||||||||| X data |||||||||||
Для файла "run1" для "X" выборки считано  dict_keys(['X_time_def', 'X_time_non_def', 'X_amp_def', 'X_amp_non_def'])
Для файла "run2" для "X" выборки считано  dict_keys(['X_time_def', 'X_time_non_def', 'X_amp_def', 'X_amp_non_def'])

||||||||||| Y data |||||||||||
Для файла "run1" для "Y" выборки считано  dict_keys(['Y_binary_def', 'Y_binary_non_def'])
Для файла "run2" для "Y" выборки считано  dict_keys(['Y_binary_def', 'Y_binary_non_def'])


In [13]:
print('|||||||||||||| X dataset ||||||||||||||||')
for key in X_dict.keys():
    print(f"|||||||||||||| file: {key}||||||||||||||||")
    for qkey in X_dict[key].keys():
        print(f"{qkey}.shape: ", X_dict[key][qkey].shape)

print('\n|||||||||||||| Y dataset ||||||||||||||||')
for key in Y_dict.keys():
    print(f"|||||||||||||| file: {key}||||||||||||||||")
    for qkey in Y_dict[key].keys():
        print(f"{qkey}.shape: ", Y_dict[key][qkey].shape)

|||||||||||||| X dataset ||||||||||||||||
|||||||||||||| file: run1||||||||||||||||
X_time_def.shape:  (513, 64, 64, 32)
X_time_non_def.shape:  (1350, 64, 64, 32)
X_amp_def.shape:  (513, 64, 64, 32)
X_amp_non_def.shape:  (1350, 64, 64, 32)
|||||||||||||| file: run2||||||||||||||||
X_time_def.shape:  (622, 64, 64, 32)
X_time_non_def.shape:  (1379, 64, 64, 32)
X_amp_def.shape:  (622, 64, 64, 32)
X_amp_non_def.shape:  (1379, 64, 64, 32)

|||||||||||||| Y dataset ||||||||||||||||
|||||||||||||| file: run1||||||||||||||||
Y_binary_def.shape:  (513,)
Y_binary_non_def.shape:  (1350,)
|||||||||||||| file: run2||||||||||||||||
Y_binary_def.shape:  (622,)
Y_binary_non_def.shape:  (1379,)


In [ ]:
print('|||||||||||||||||| X data ||||||||||||||||||')
print('|||||||||||||||||| augment train data ||||||||||||||||||')
for key in X_dict.keys():
    print(f"|||||||||||||| file: {key}||||||||||||||||")
    for qkey in X_dict[key].keys():
        print(f"|||||||||||||| dataset part: {qkey}||||||||||||||||")
        X_dict[key][qkey] = dp.augment_data(X_dict[key][qkey])      

print('|||||||||||||||||| Y data ||||||||||||||||||')                
for key in Y_dict.keys():
    print(f"|||||||||||||| file: {key}||||||||||||||||")
        
    print(f"|||||||||||||| dataset part: Y_binary_def||||||||||||||||")
    print(f'Orig arr shape: {Y_dict[key]["Y_binary_def"].shape}')
    Y_dict[key]["Y_binary_def"] = np.ones(X_dict[key]['X_time_def'].shape[0])
    print(f'Result arr shape: {Y_dict[key]["Y_binary_def"].shape}')

    print(f"|||||||||||||| dataset part: Y_binary_non_def||||||||||||||||")
    print(f'Orig arr shape: {Y_dict[key]["Y_binary_non_def"].shape}')
    Y_dict[key]["Y_binary_non_def"] = np.zeros(X_dict[key]['X_time_non_def'].shape[0])
    print(f'Result arr shape: {Y_dict[key]["Y_binary_non_def"].shape}')

In [14]:
print('|||||||||||||| X dataset ||||||||||||||||')
for key in X_dict.keys():
    print(f"|||||||||||||| file: {key}||||||||||||||||")
    for qkey in X_dict[key].keys():
        print(f"{qkey}.shape: ", X_dict[key][qkey].shape)

print('\n|||||||||||||| Y dataset ||||||||||||||||')
for key in Y_dict.keys():
    print(f"|||||||||||||| file: {key}||||||||||||||||")
    for qkey in Y_dict[key].keys():
        print(f"{qkey}.shape: ", Y_dict[key][qkey].shape)

|||||||||||||| X dataset ||||||||||||||||
|||||||||||||| file: run1||||||||||||||||
X_time_def.shape:  (513, 64, 64, 32)
X_time_non_def.shape:  (1350, 64, 64, 32)
X_amp_def.shape:  (513, 64, 64, 32)
X_amp_non_def.shape:  (1350, 64, 64, 32)
|||||||||||||| file: run2||||||||||||||||
X_time_def.shape:  (622, 64, 64, 32)
X_time_non_def.shape:  (1379, 64, 64, 32)
X_amp_def.shape:  (622, 64, 64, 32)
X_amp_non_def.shape:  (1379, 64, 64, 32)

|||||||||||||| Y dataset ||||||||||||||||
|||||||||||||| file: run1||||||||||||||||
Y_binary_def.shape:  (513,)
Y_binary_non_def.shape:  (1350,)
|||||||||||||| file: run2||||||||||||||||
Y_binary_def.shape:  (622,)
Y_binary_non_def.shape:  (1379,)


In [15]:
X_time_test = np.concatenate([X_dict['run1']['X_time_def'],
                                X_dict['run1']['X_time_non_def'],
                                X_dict['run2']['X_time_def'],
                                X_dict['run2']['X_time_non_def']],
                                    axis=0)
X_dict['run1'].pop('X_time_def',0)
X_dict['run1'].pop('X_time_non_def',0)
X_dict['run2'].pop('X_time_def',0)
X_dict['run2'].pop('X_time_non_def',0)

X_amp_test = np.concatenate([X_dict['run1']['X_amp_def'],
                                X_dict['run1']['X_amp_non_def'],
                                X_dict['run2']['X_amp_def'],
                                X_dict['run2']['X_amp_non_def']],
                                    axis=0)

X_dict['run1'].pop('X_amp_def',0)
X_dict['run1'].pop('X_amp_non_def',0)
X_dict['run2'].pop('X_amp_def',0)
X_dict['run2'].pop('X_amp_non_def',0)


Y_binary_test= np.concatenate([Y_dict['run1']['Y_binary_def'],
                                Y_dict['run1']['Y_binary_non_def'],
                                Y_dict['run2']['Y_binary_def'],
                                Y_dict['run2']['Y_binary_non_def']],
                                    axis=0)

Y_dict['run1'].pop('Y_binary_def',0)
Y_dict['run1'].pop('Y_binary_non_def',0)
Y_dict['run2'].pop('Y_binary_def',0)
Y_dict['run2'].pop('Y_binary_non_def',0)

array([0., 0., 0., ..., 0., 0., 0.])

In [16]:
# очистка памяти от лишнего
        
X_dict.clear()
Y_dict.clear()

In [17]:
print('X_time_test.shape: ', X_time_test.shape)
print('X_amp_test.shape: ', X_amp_test.shape)
print('Y_binary_test.shape', Y_binary_test.shape)
print('\n')

X_time_test.shape:  (3864, 64, 64, 32)
X_amp_test.shape:  (3864, 64, 64, 32)
Y_binary_test.shape (3864,)




# CNN с 1 входом по времени

In [18]:
model = keras.models.load_model('Networks//CNN//NetWork_(64x32+64x32)_to(1)_(test_0dot0).h5')

In [19]:
# тест модели
model.evaluate([X_time_test,X_amp_test], Y_binary_test, batch_size = 1)

3864/3864 [==============================] - 110s 28ms/step - loss: 0.1996


0.19961786270141602